In [1]:
print("ok")

ok


In [36]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from pinecone.grpc import PineconeGRPC as Pinecone

In [22]:
import os
PINECONE_API_KEY = os.getenviron('PINECONE_API_KEY')

In [4]:
#extract data from pdf
def load_pdf(data):
    loader = DirectoryLoader(data,glob="*.pdf",loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [8]:
extracted_data = load_pdf("../data")

In [12]:
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [13]:
text_chunks = text_split(extracted_data)

In [14]:
len(text_chunks)

7020

In [17]:
print(text_chunks[0].page_content)

TheGALE
ENCYCLOPEDIA
ofMEDICINE
SECOND EDITION


In [18]:
#download embedding model
def download_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [19]:
embeddings = download_embeddings()

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]


In [ ]:
query_result = embeddings.embed_query("heelo")
print(query_result)

In [26]:
pc = Pinecone(api_key=PINECONE_API_KEY)

In [27]:
index_name="medical-chat-bot"

In [31]:
index = pc.Index(host="https://medical-chat-bot-dl2ip24.svc.aped-4627-b74a.pinecone.io")

In [37]:
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [39]:
texts = [doc.page_content for doc in text_chunks]

In [ ]:
vectors = [(f'vec_{i}', embeddings[i].tolist(), {"text": texts[i]}) for i in range(len(texts))]
index = pinecone.Index(index_name)

upsert_response = index.upsert(
    vectors=vectors,
    namespace="example-namespace"
)